In [72]:
# importing the requests library 
import requests 
import pandas as pd
from random import sample 
import matplotlib
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns                                                             
import sys
import scipy
import nltk
from nltk.stem import WordNetLemmatizer 
from spellchecker import SpellChecker
from numpy import linalg as LA

After manually classifying 150 function I noticed that most common mistake are: word ordering and word mispelling. Another thing I noticed where that each miss written function ussually was in a pham with the correct approved function name. With these two realizations I want to make a spell checker system that takes into account common error and genetic makeup.

#### Steps to Correct to Building Bag of Words Vectorized Database
1. Make a corpus of words found in approved functions
2. Token and lemminize that corpus
3. Make a vector representation of each approved function

#### Steps to Correct Non-Approved Function Names
1. Pre-proccessing: Delete words that are non-functional, ie "punative", "protien", "domain"
2. Pre-proccessing: Tokenize each word
3. Pre-proccessing: Convert acroymns to full words based on known acronyms
4. Pre-proccessing: Identify known unapproved functions (from SEA-PHAGES list) and covert to known functions
5. Pre-proccessing: Spell correct to attain words found in function corpus
6. Use bag of words technique to create a input vector
7. Use nearest neighbors of approved functions in thier PHAM to determine approved function


# Building Bag of Words Vectorized Database


### 1. Make a corpus of words found in approved functions

In [91]:
df_approved_functions = pd.read_csv("Approved_Functions.csv")
df_approved_functions = df_approved_functions.dropna(subset=["Approved Function"])
df_approved_functions.head()

,Approved Function,Unapproved Functions
0,"terminase, small subunit",TerS
1,terminase,NaN
2,"terminase, large subunit",TerL
3,"terminase, large subunit (ATPase domain)",NaN
4,"terminase, large subunit (nuclease domain)",NaN


In [143]:
approved_functions = list(df_approved_functions["Approved Function"])
approved_functions = [i.lower() for i in approved_functions]
approved_functions

['terminase, small subunit',
 'terminase',
 'terminase, large subunit',
 'terminase, large subunit (atpase domain)',
 'terminase, large subunit (nuclease domain)',
 'portal protein',
 'portal and muf-like fusion protein',
 'scaffolding protein',
 'capsid maturation protease',
 'major capsid protein',
 'minor capsid protein',
 'muf-like minor capsid protein',
 'capsid decoration protein, lamd-like',
 'capsid maturation protease and muf-like fusion protein',
 'major capsid and protease fusion protein',
 'head fiber protein',
 'head-to-tail adaptor',
 'head-to-tail stopper',
 'tail terminator',
 'major tail protein',
 'tail assembly chaperone',
 'tape measure protein',
 'minor tail protein',
 'minor tail protein, d-ala-d-ala carboxypeptidase',
 'tail sheath protein',
 'tail fiber',
 'tailspike protein',
 'tail needle protein',
 'baseplate j protein',
 'tail tube protein',
 'baseplate wedge protein',
 'capsid decoration protein',
 'lysin a',
 'lysin a, protease m15 domain',
 'lysin a, prot

### 2. Token and lemminize that corpus

In [93]:
non_unique_corpus = []
for function in approved_functions:
    non_unique_corpus += nltk.word_tokenize(function) 

unique_corpus = np.sort(np.unique(non_unique_corpus)) 
unique_corpus = [i.lower() for i in unique_corpus]
unique_corpus[1:10]

corpus = unique_corpus

In [94]:
len(corpus)

318

### 3. Make a vector representation of each approved function

In [95]:
function_vectors = {}
for func in approved_functions:
    function_vectors[func] = [0]*(len(corpus)+1)
    for token in nltk.word_tokenize(func):
        if token.lower() in corpus:
#             print(token, corpus[corpus.index(token.lower())])
            function_vectors[func][corpus.index(token.lower())] = 1

In [115]:
# FILTER BASED ON COUNT
COUNT_FILTER = [0]*(len(corpus)+1)
for c in corpus:
    COUNT_FILTER[corpus.index(c)] = 1/non_unique_corpus.count(c)

In [117]:
COUNT_FILTER[corpus.index("domain")]

0.058823529411764705

In [122]:
np.multiply([1,0,1],[1,1,3])

array([1, 0, 3])

### Create known conversion list

In [96]:
unapproved_functions = []
conversion_list = {}
for _, row in df_approved_functions.dropna(subset=["Approved Function", "Unapproved Functions"]).iterrows():
    for i in row["Unapproved Functions"].split(", "):
        conversion_list[i.lower()] = row["Approved Function"]
        unapproved_functions += [i.lower()]
        
conversion_list

{'ters': 'terminase, small subunit',
 'terl': 'terminase, large subunit',
 'head to tail connector': 'portal protein',
 'scaffold': 'scaffolding protein',
 'capsid': 'major capsid protein',
 'capsid morphogenesis protein': 'capsid maturation protease and MuF-like fusion protein',
 'head-to-tail connector': 'head-to-tail adaptor',
 'head-to-tail connector complex protein': 'head-to-tail adaptor',
 'major tail subunit': 'major tail protein',
 'tail scaffolding protein': 'tail assembly chaperone',
 'tape measure': 'tape measure protein',
 'tmp': 'tape measure protein',
 'tapemeasure': 'tape measure protein',
 'tail fiber-like protein': 'minor tail protein',
 'collagen-like': 'minor tail protein',
 'glycine rich': 'minor tail protein',
 'head decoration protein': 'capsid decoration protein',
 'lysa': 'lysin A',
 'endolysin a': 'lysin A',
 'lysb': 'lysin B',
 'endolysin b': 'lysin B',
 'repressor': 'immunity repressor',
 'excisionase': 'excise',
 'xis': 'excise',
 'rdf': 'recombination dire

In [97]:
synomym_list = {"head": "capsid",
                "connector": "adaptor", 
                "single-stranded dna": "ssdna",
                "hth": "helix-turn-helix",
                "cro": "cro (control of repressor’s operator)"
               }
for key, value in conversion_list.items():
    if len(nltk.word_tokenize(key)) == 1:
        synomym_list[key] = value
    
synomym_list

{'head': 'capsid',
 'connector': 'adaptor',
 'single-stranded dna': 'ssdna',
 'hth': 'helix-turn-helix',
 'ters': 'terminase, small subunit',
 'terl': 'terminase, large subunit',
 'scaffold': 'scaffolding protein',
 'capsid': 'major capsid protein',
 'tmp': 'tape measure protein',
 'tapemeasure': 'tape measure protein',
 'collagen-like': 'minor tail protein',
 'lysa': 'lysin A',
 'lysb': 'lysin B',
 'repressor': 'immunity repressor',
 'excisionase': 'excise',
 'xis': 'excise',
 'rdf': 'recombination directionality factor',
 'pnk': 'polynucleotide kinase',
 'lsr2': 'Lsr2-like DNA bridging protein',
 'ku': 'Ku-like dsDNA break-binding protein',
 'whib': 'WhiB family transcription factor',
 'thyx': 'ThyX-like thymidylate synthase',
 'dr': 'dihydrofolate reductase',
 'mazg': 'MazG-like nucleotide pyrophosphohydrolase',
 'erf': 'ssDNA binding protein, ERF family',
 'clp': 'ClpP-like protease',
 'nrdh': 'NrdH-like glutaredoxin',
 'ro': 'Ro-like RNA binding protein',
 'pura': 'adenylosuccinat

#### There are some components that are more identifiable than others

In [98]:
# fig, ax1 = plt.subplots(figsize = (20,6))
# plt.hist(func_units, bins = np.unique(func_units))
# plt.xticks([i if func_units.count(i) > 1 else "" for i in np.unique(func_units)], rotation='vertical')
# plt.show()

In [99]:
# ident_vocab = []
# common_vocab = []
# for i in np.unique(func_units):
#     if func_units.count(i) == 1:
#         ident_vocab += [i]
#     elif func_units.count(i) > 1:
#         common_vocab += [i]

In [100]:
import json

spell_dict = {}
for i in unique_corpus:
    spell_dict[i] = non_unique_corpus.count(i)
    
for i in synomym_list.keys():
    spell_dict[i] = 1
    
    
with open('functions.json', 'w') as fp:
    json.dump(spell_dict, fp)
    
spell_dict

{"'": 1,
 '(': 10,
 ')': 10,
 ',': 35,
 '1': 2,
 '2': 2,
 '5': 1,
 'a': 10,
 'aaa-atpase': 1,
 'abortive': 2,
 'acetyltransferase': 1,
 'acid': 1,
 'adaptor': 1,
 'adda-like': 1,
 'adenylate': 2,
 'adenylosuccinate': 2,
 'adenylyltransferase': 1,
 'adp-ribosyl': 1,
 'adp-ribosyltransferase': 3,
 'alpha': 1,
 'amidase': 1,
 'amidotransferase': 1,
 'amine': 1,
 'aminotransferase': 1,
 'and': 4,
 'antirepressor': 1,
 'antirestriction': 2,
 'antitoxin': 4,
 'arda-like': 1,
 'arsenate': 1,
 'asc-1': 1,
 'assembly': 1,
 'atp': 1,
 'atpase': 1,
 'b': 1,
 'band-7': 1,
 'baseplate': 2,
 'beta': 1,
 'binding': 13,
 'biosynthesis': 1,
 'break': 1,
 'break-binding': 2,
 'bridging': 1,
 'brnt-like': 1,
 'c39': 1,
 'capsid': 1,
 'carboxylate': 1,
 'carboxylesterase': 1,
 'carboxypeptidase': 1,
 'cas4': 1,
 'cassete-like': 1,
 'chain': 2,
 'chaperone': 1,
 'chaperonin': 1,
 'clamp': 1,
 'clpp-like': 1,
 'cluster': 1,
 'coactivator': 1,
 'cobalamin': 1,
 'cobt-like': 1,
 'control': 1,
 'cro': 1,
 'cyc

In [101]:
# https://readthedocs.org/projects/pyspellchecker/downloads/pdf/latest/

Examples of errors:

1. "helix-turn-helix dna binding domain protein" this happens when unimportant words are added
"helix-turn-helix dna binding domain protein"

2. "terminase large subunit" happens when punctuation is forgotten

In [161]:
spell = SpellChecker(local_dictionary="functions.json", distance=3)
def find_approved_function(function, pham_members):
    print("input:",function)
    if function in conversion_list:
        function = conversion_list[function]
    # clean function
    #     function = "baseplate J domain faseto protein"
    cleaned_function = []
    # tokens = []
    for i in nltk.word_tokenize(function): #reca
        if spell.word_probability(i.lower()+"-like", 1) > spell.word_probability(i.lower(), 1):
            corrected_token = spell.correction(i.lower()+"-like")
        else:
            corrected_token = spell.correction(i.lower())
            
        if corrected_token in synomym_list:  
            corrected_token = synomym_list[corrected_token]

        corrected_token = nltk.word_tokenize(corrected_token)
            
        cleaned_function += corrected_token

    print("cleaned function:", cleaned_function)

    # create bag of words vector
    vec = [0]*(len(corpus)+1)
    for i in cleaned_function:
        if i in corpus:
            vec[corpus.index(i)] = 1
        else:
            vec[-1] = 1

    # find nearest neighbor
    min_value = float('inf')
    min_key = ""
    for key in pham_members:
        func_vec = function_vectors[key]
        dist = LA.norm(np.multiply(COUNT_FILTER,func_vec)-np.multiply(COUNT_FILTER,vec))
        if dist < min_value:
            min_value = dist
            min_key = key
            
    print("conversion:", min_key, min_value) 
    return min_key

In [140]:
df_genes = pd.read_csv("all_phage_genes.csv")
df_genes = df_genes.applymap(str)
df_genes["GeneNumber"] = df_genes["GeneNumber"].apply(pd.to_numeric)
df_genes["Function"] = df_genes["Function"].str.lower()
df_genes = df_genes.applymap(str)
df_genes.describe(include = 'all')
df_genes.head()

,GeneID,HostStrain,Cluster,Pham,Function,Translation,Orientation,PhageName,GeneNumber
0,20ES_CDS_1,Mycobacterium,A2,36676,nan,MYGTRSSAFWASQPGKFDVLNLRMTFPSTSAHEIPDLTATDFVPEN...,F,20ES,1
1,20ES_CDS_10,Mycobacterium,A2,34452,lysin b,MSLQVGSSGELVNRWIRVMKARFASYAGKLKEDGYFGLDDKAVQQE...,F,20ES,10
2,20ES_CDS_11,Mycobacterium,A2,34196,terminase,MSLENHHPELAPSPPHIIGPSWQRTVDGSWHLPDPKMTLGWGVLKW...,F,20ES,11
3,20ES_CDS_12,Mycobacterium,A2,37970,portal protein,MTAPLPGQEEIPDPAIARDEMISAFDDAVKNLKINTSYYEAERRPE...,F,20ES,12
4,20ES_CDS_13,Mycobacterium,A2,21454,capsid maturation protease,MITAAVAAYVQRFASMFTGPALSLGEWARFLQTLFPEVQRRYAQAA...,F,20ES,13


In [134]:
non_corresponding = []
for index, row in df_genes.iterrows():
    function = row["Function"]
    if function not in approved_functions and function != "nan":
        print(row["Function"])
        non_corresponding.append(row["Function"])

scaffold protein
major capsid subunit
major tail subunit
minor tail subunit
minor tail subunit
minor tail subunit
para
parb
terminase small subunit
thyx-like protein
endovii
esterase/lipase
hth binding protein
dnab-like helicase
recb-like protein
small terminase subunit
capsid protein
virion protein
major tail subunit
tapemeasure protein
virion protein
minor tail subunit
virion protein
large terminase subunit
virion protein
integrase
repressor
hth binding protein
dnaq
rece like protein
rect
putative hth binding protein
ruvc
whib
parb-like protein
capsid protein
major tail subunit
queuine-trna ribosyltransferase
tapemeasure protein
minor tail subunit
minor tail subunit
minor tail subunit
terminase small subunit
hth binding protein
terminase large subunit
putative holin
type iii restriction helicase
primase/helicase
ruvc
pe/ppe-like protein
terminase large subunit
scaffold protein
capsid protein
tail protein
minor tail subunit
minor tail subunit
para
hth dna binding protein
terminase sma

helix-turn-helix dna-binding domain protein
terminase large subunit
arda-like antirestriction protein
terminase large subunit
terminase small subunit
helix-turn-helix dna-binding protein
helix-turn-helix dna-binding protein
helix-turn-helix dna-binding domain protein
pura-like adenylosuccinate synthetase
helix-turn-helix dna-binding domain protein
dna recombinase
terminase small subunit
helix-turn-helix dna binding domain protein
terminase large subunit
integrase
rele-like toxin
relb-like antitoxin
cro protein
helix-turn-helix dna binding domain protein
mpme 1 protein
merr-like hth dna binding protein
helix-turn-helix dna binding protein
rect-like ssdna binding protein
helix-turn-helix dna binding protein
dna polymerase iii subunit
putative head decoration, structural
putative tail sheath protein
tapemeasure protein
putative minor tail protein
putative minor tail protein
putative baseplate j-like protein
putative minor tail protein
peptidyl-trna hydrolase
d-ala-d-ala carboxypeptidase
a

integrase
hth dna binding protein
dcmp deaminase
thyx
hth dna binding protein
esterase/lipase
dnab-like helicase
parb-like nuclease domain protein
helix-turn-helix dna binding protein
helix-turn-helix dna binding protein
mazg-like nucleotide pyrophosphate
nucleotidyltransferase
exonuclease vii
mre11 double-strand break repair endo/exonuclease
rect-like ssdna binding protein
helix-turn-helix dna binding protein
helix-turn-helix dna binding protein
queuine-trna ribosyltransferase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
integrase
cro protein
hth dna binding protein
hth dna binding domain protein
whib
dna polymerase iii subunit
terminase small subunit
terminase large subunit
resolvase
dna binding domain protein
hicb-like antitoxin
hica-like toxin
endonuclease
dsdna helicase
parb-like nuclease domain protein
helix-turn-helix dna binding protein
helix-turn-helix dna binding protein
helix-turn-helix dna-binding domain protein
pura-like adenylosu

terminase small subunit
lysm-like endolysin
terminase large subunit
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
integrase
head-to-tail connector complex protein
dna recombinase
parb-like nuclease domain protein
helix-turn-helix dna binding protein
helix-turn-helix dna binding protein
parb-like nuclease domain protein
ribbon-helix-helix dna binding protein
tapemeasure protein
peptidyl-trna hydrolase
dsdna helicase
dnaj-like chaperonin
dna polymerase iii alpha subunit
dna recombinase
resolvase
nicotinamide ribosyltransferase
homing endonuclease domain protein
pura-like adenylosuccinate synthetase
nucleotidyltransferase
band-7-like membrane protein
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
lysm-like endolysin
head-to-tail connector complex protein
integrase
helix-turn-helix dna-binding domain protein
helix-turn-helix dna-binding domain protein
helix-turn-helix dna binding protein
helix-turn-helix dna

recb-like exonuclease
helix-turn-helix dna binding protein
dnaq-like dna polymerase iii subunit
paps reductase-like protein
terminase small subunit
terminase large subunit
helix-turn-helix dna binding domain protein
capsid morphogenesis protein
helix-turn-helix dna binding domain protein
dnaj-like chaperonin
asc-1 transcription coactivator
helix-turn-helix dna binding domain protein
parb-like nuclease domain protein
helix-turn-helix dna binding protein
helix-turn-helix dna binding protein
parb-like nuclease domain protein
ribbon-helix-helix dna binding protein
hica-like toxin
dna recombinase
parb-like nuclease domain protein
queuine-trna ribosyltransferase
quec-like queosine biosynthesis protein
qued-like queosine biosynthesis protein
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding protein
quee-like queosine biosynthesis protein
ribbon-helix-helix dna binding domain protein
hica-like toxin
dna recombinase
d-ala-d-ala carboxypeptidase
dnaj-like chaperonin
rusa-l

...

terminase large subunit
hypothetical function
cas4-like exonuclease
head-to-tail connector complex protein
head-to-tail connector complex protein
head-to-tail connector complex protein
terminase small subunit
hth dna binding protein
terminase large subunit
hth dna binding protein
integrase
queuine trna-ribosyltransferase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
phospholipase
helix-turn-helix dna binding domain protein
lysm-like endolysin
helix-turn-helix dna binding domain protein
terminase small subunit
dnae-like dna polymerase iii
terminase large subunit
terminase large subunit
helix-turn-helix dna-binding domain protein
terminase small subunit
hth dna binding protein
dna polymerase iii subunit
dna recombinase
helix-turn-helix dna binding domain protein
head-to-tail connector protein
hth dna binding protein
membrane domain protein
membrane domain protein
dna bridging protein
membrane domain protein
integrase
membrane domain protein
membr

dna methyltransferase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
nicotinamide riboside transporter
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
parb-like nuclease domain protein
glycerophosphodiester phosphodiesterase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
rect-like ssdna binding protein
putative zinc peptidase
major tail subunit
tapemeasure
queuine trna-ribosyltransferase
integrase y-int
hth dna binding protein
hth dna binding protein
whib-like protein
terminase small subunit
dnaq-like protein
terminase large subunit
nrdh-like protein
rusa-like protein
putative pe family protein
sprt-like protein
rtcb-like protein
ycfa-like protein
queuine trna-ribosyltransferase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding protein
helix-turn-helix dna-binding domain protein
pura-like adenylosucc

helix-turn-helix dna binding domain protein
helix-turn-helix dna-binding domain protein
helix-turn-helix dna-binding domain protein
terminase large subunit
tapemeasure protein
terminase small subunit
dnab-like dna helicase
helix-turn-helix dna binding domain protein
glycosyl hydrolase
dnaq-like dna polymerase iii subunit
parb-like nuclease domain protein
head-to-tail connector protein
integrase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
dsdna helicase
helix-turn-helix dna-binding domain protein
pura-like adenylosuccinate synthetase
helix-turn-helix dna-binding domain protein
helix-turn-helix dna-binding domain protein
pura-like adenylosuccinate synthetase
helix-turn-helix dna-binding domain protein
single-stranded dna binding protein
dna polymerase iii alpha
dna polymerase iii alpha
dna recombinase
dna binding domain protein
terminase large subunit
terminase associated hnh endonuclease
phosphohydro

head-to-tail connector protein
integrase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
dsdna helicase
terminase large subunit
head-to-tail connector protein
head-to-tail connector protein
tapemeasure protein
helix-turn-helix dna binding domain protein
terminase small subunit
terminase small subunit
nucleotidyltransferase
cro protein
terminase large subunit
erf family ssdna binding protein
rusa-like resolvase
dnaq-like dna polymerase iii subunit
terminase small subunit
terminase large subunit
integrase
helix-turn-helix dna binding domain protein
cro protein
helix-turn-helix dna binding domain protein
resolvase
helix-turn-helix dna binding domain protein
helix-turn-helix dna binding domain protein
queuine trna-ribosyltransferase
helix-turn-helix dna binding domain protein
hth dna binding protein
helix-turn-helix dna binding domain protein
terminase small subunit
integrase
helix-turn-helix dna binding do

rusa-like resolvase
helix-turn-helix dna binding domain protein
dnaq-like dna polymerase iii subunit
lysa
terminase large subunit
tapemeasure protein
dcmp deaminase
hth domain protein
terminase small subunit
endovii
sprt-like protein
whib transcriptional regulator
terminase large subunit
minor tail subunit
minor tail subunit
minor tail subunit
d-ala-d-ala-carboxypeptidase
minor tail subunit
minor tail subunit
hth dna binding protein
dna polymerase iii
ku protein
parb-like protein
d-ala-d-ala carboxypeptidase
dnaj-like chaperonin
rusa-like resolvase
pura-like adenylosuccinate synthetase
band-7-like membrane protein
helix-turn-helix dna-binding domain protein
helix-turn-helix dna-binding domain protein
helix-turn-helix dna-binding domain protein
parb-like nuclease domain protein
lysm-like endolysin
terminase small subunit
terminase large subunit
lysin a, peptidase domain
helix-turn-helix dna binding domain protein
integrase
dna binding domain protein
helix-turn-helix dna binding domain p

In [135]:
unique, counts = np.unique(non_corresponding, return_counts=True)

In [127]:
zipped = zip(unique, counts) 
  
# Converting to list 
zipped = list(zipped) 
  
unapproved_functions = sorted(zipped, key = lambda x: x[1], reverse=True)

In [162]:
for target_function, count in unapproved_functions[:100]:
    phams = list(df_genes[df_genes["Function"] == target_function]["Pham"])
    official_functions = {}
    for pham in phams:
        functions_pham = list(df_genes[df_genes["Pham"] == pham]["Function"])
        for func in functions_pham:
            if func in approved_functions:
                if func not in official_functions:
                    official_functions[func] = 1
                else:
                    official_functions[func] += 1
    
    out_function = find_approved_function(target_function, official_functions.keys())
    
    if out_function in official_functions:
        print("APPROVED")
    else:
        print("NOT FOUND")
    print()


input: helix-turn-helix dna binding domain protein
cleaned function: ['helix-turn-helix', 'dna', 'binding', 'domain', 'protein']
conversion: helix-turn-helix dna binding domain 0.02127659574468085
APPROVED

input: terminase large subunit
cleaned function: ['terminase', 'large', 'subunit']
conversion: terminase, large subunit 0.02857142857142857
APPROVED

input: helix-turn-helix dna binding protein
cleaned function: ['helix-turn-helix', 'dna', 'binding', 'protein']
conversion: helix-turn-helix dna binding domain 0.06255318648110038
APPROVED

input: terminase small subunit
cleaned function: ['terminase', 'small', 'subunit']
conversion: terminase, small subunit 0.02857142857142857
APPROVED

input: integrase
cleaned function: ['integrase']
conversion: tyrosine integrase 0.3333333333333333
APPROVED

input: helix-turn-helix dna-binding domain protein
cleaned function: ['helix-turn-helix', 'dna-binding', 'domain', 'protein']
conversion: helix-turn-helix dna binding domain 0.09088571096733604


input: n-acetyltransferase
cleaned function: ['acetyltransferase']
conversion: acetyltransferase 0.0
APPROVED

input: nrdh
cleaned function: ['nrdh-like', 'glutaredoxin']
conversion: nrdh-like glutaredoxin 0.0
APPROVED

input: ssdna binding domain protein
cleaned function: ['ssdna', 'binding', 'domain', 'protein']
conversion: ssdna binding protein 0.058823529411764705
APPROVED

input: dna polymerase iii (alpha)
cleaned function: ['dna', 'polymerase', 'iii', '(', 'alpha', ')']
conversion: dnae-like dna polymerase iii (alpha) 1.0
APPROVED

input: recb-like exonuclease
cleaned function: ['recb-like', 'exonuclease']
conversion: exonuclease 1.0
APPROVED

input: thyx
cleaned function: ['thyx-like', 'thymidylate', 'synthase']
conversion: thyx-like thymidylate synthase 0.0
APPROVED

input: galactosaminyltransferase
cleaned function: ['galactosaminyltransferase']
conversion: glycosyltransferase 1.0
APPROVED

input: lysm domain protein
cleaned function: ['lysm-like', 'domain', 'protein']
convers